In [2]:
import os
import zipfile
import numpy as np

In [4]:
f_name = 'Lab2.zip'
print('File size', os.path.getsize(f_name) / 1024 / 1024, 'MB')

File size 7.843085289001465 MB


In [5]:
# To unzip the archive 'features.zip' in the current directory

with zipfile.ZipFile(f_name, 'r') as fd:
    fd.extractall('.')

In [7]:
f_name2 = 'pybsds.zip'
print('File size', os.path.getsize(f_name2) / 1024 / 1024, 'MB')

File size 0.01936817169189453 MB


In [8]:
# To unzip the archive 'features.zip' in the current directory

with zipfile.ZipFile(f_name2, 'r') as fd:
    fd.extractall('.')

# Original countour_demo.py

In [15]:
# Code from Saurabh Gupta
from tqdm import tqdm
import os, sys, numpy as np, cv2
sys.path.insert(0, 'pybsds')
from scipy import signal
from skimage.util import img_as_float
from skimage.io import imread
from Lab2.contours.pybsds.bsds_dataset import BSDSDataset
from Lab2.contours.pybsds import evaluate_boundaries
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import json
import time
from absl import flags, app
from Lab2.contours.contour_plot import display_results

from Lab2.contours.contour_solve import compute_edges_dxdy

#FLAGS = flags.FLAGS
#flags.DEFINE_string('output_dir', 'output/demo', 'directory to save results.')
#flags.DEFINE_string('imset', 'val', 'image set to use for testing')

N_THRESHOLDS = 99

def get_imlist(imset):
  imlist = np.loadtxt(f'Lab2/contours/data/{imset}/imlist')
  return imlist.astype(np.int)

def detect_edges(imlist, fn, out_dir):
  total_time = 0
  for imname in tqdm(imlist):
    I = cv2.imread(os.path.join('Lab2/contours/data', 'val', 'images', str(imname)+'.jpg'))
    start_time = time.time()
    mag = fn(I)
    total_time += time.time() - start_time
    out_file_name = os.path.join(out_dir, str(imname)+'.png')
    cv2.imwrite(out_file_name, mag)
  return total_time / len(imlist)

def load_gt_boundaries(imname):
    gt_path = os.path.join('Lab2/contours/data', 'val', 'groundTruth', '{}.mat'.format(imname))
    return BSDSDataset.load_boundaries(gt_path)

def load_pred(output_dir, imname):
    pred_path = os.path.join(output_dir, '{}.png'.format(imname))
    return img_as_float(imread(pred_path))

def save_results(out_file_name, threshold_results, overall_result, runtime):
  res = np.array(threshold_results)
  recall = res[:,1]
  precision = res[recall>0.01,2]
  recall = recall[recall>0.01]
  threshold = overall_result.threshold
  f1 = overall_result.f1
  best_f1 = overall_result.best_f1
  area_pr = overall_result.area_pr
  np.savez(out_file_name, precision=precision, recall=recall,
           area_pr=area_pr, best_f1=best_f1, f1=f1, runtime=runtime,
           threshold=threshold)
  return threshold, precision, recall, f1, best_f1, area_pr

def main():
  fn = compute_edges_dxdy
  imlist = get_imlist('val')

  output_dir = 'output/demo'
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  bench_dir = os.path.join('output/demo', 'bench')
  if not os.path.exists(bench_dir):
    os.makedirs(bench_dir)

  print('Running detector:')
  runtime = detect_edges(imlist, fn, bench_dir)

  print('Evaluating:')
  sample_results, threshold_results, overall_result = \
    evaluate_boundaries.pr_evaluation(N_THRESHOLDS, imlist,
                                      load_gt_boundaries,
                                      lambda x: load_pred(bench_dir, x),
                                      fast=True, progress=tqdm)
  print('Save results:')
  out_file_name = os.path.join('output/demo', 'metrics.npz')
  threshold, precision, recall, f1, best_f1, area_pr = \
    save_results(out_file_name, threshold_results, overall_result, runtime)

  fig = plt.figure(figsize=(6,6))
  ax = fig.gca()
  display_results(ax, 'output/demo', threshold, precision, recall, f1, best_f1, area_pr)
  print('{:>24s}: {:<10.6f}'.format('runtime (in seconds)', runtime))

  fig.savefig(os.path.join(output_dir + '_pr.pdf'), bbox_inches='tight')

#if __name__ == '__main__':
 # app.run(main)


In [16]:
main()

<ipython-input-15-576415080106>:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return imlist.astype(np.int)


Running detector:


100%|██████████| 50/50 [00:00<00:00, 61.41it/s]


Evaluating:


100%|██████████| 50/50 [02:07<00:00,  2.55s/it]


Save results:
               threshold: 0.140000  
    overall max F1 score: 0.518496  
    average max F1 score: 0.556084  
                      AP: 0.426090  
    runtime (in seconds): 0.009928  


# Question 3  - Gaussian Blur

**contour_solve.py**

In [ ]:
import numpy as np
from scipy import signal
import cv2

def compute_edges_dxdy(I):
  """Returns the norm of dx and dy as the edge response function."""
  I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
  I = I.astype(np.float32)/255.

  I_smoothened = cv2.GaussianBlur(I, (0, 0), sigmaX = 0 .5, sigmaY = 0.5)

  dx = signal.convolve2d(I, np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]), mode='same')
  dy = signal.convolve2d(I, np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]).T, mode='same')
  mag = np.sqrt(dx**2 + dy**2)
  mag = mag / 1.5
  mag = mag * 255.
  mag = np.clip(mag, 0, 255)
  mag = mag.astype(np.uint8)
  return mag


In [25]:
# Code from Saurabh Gupta
from tqdm import tqdm
import os, sys, numpy as np, cv2
sys.path.insert(0, 'pybsds')
from scipy import signal
from skimage.util import img_as_float
from skimage.io import imread
from Lab2.contours.pybsds.bsds_dataset import BSDSDataset
from Lab2.contours.pybsds import evaluate_boundaries
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import json
import time
from absl import flags, app
from Lab2.contours.contour_plot import display_results

from Lab2.contours.contour_solve import compute_edges_dxdy

#FLAGS = flags.FLAGS
#flags.DEFINE_string('output_dir', 'output/demo', 'directory to save results.')
#flags.DEFINE_string('imset', 'val', 'image set to use for testing')

N_THRESHOLDS = 99

def get_imlist(imset):
  imlist = np.loadtxt(f'Lab2/contours/data/{imset}/imlist')
  return imlist.astype(np.int)

def detect_edges(imlist, fn, out_dir):
  total_time = 0
  for imname in tqdm(imlist):
    I = cv2.imread(os.path.join('Lab2/contours/data', 'val', 'images', str(imname)+'.jpg'))
    start_time = time.time()
    mag = fn(I)
    total_time += time.time() - start_time
    out_file_name = os.path.join(out_dir, str(imname)+'.png')
    cv2.imwrite(out_file_name, mag)
  return total_time / len(imlist)

def load_gt_boundaries(imname):
    gt_path = os.path.join('Lab2/contours/data', 'val', 'groundTruth', '{}.mat'.format(imname))
    return BSDSDataset.load_boundaries(gt_path)

def load_pred(output_dir, imname):
    pred_path = os.path.join(output_dir, '{}.png'.format(imname))
    return img_as_float(imread(pred_path))

def save_results(out_file_name, threshold_results, overall_result, runtime):
  res = np.array(threshold_results)
  recall = res[:,1]
  precision = res[recall>0.01,2]
  recall = recall[recall>0.01]
  threshold = overall_result.threshold
  f1 = overall_result.f1
  best_f1 = overall_result.best_f1
  area_pr = overall_result.area_pr
  np.savez(out_file_name, precision=precision, recall=recall,
           area_pr=area_pr, best_f1=best_f1, f1=f1, runtime=runtime,
           threshold=threshold)
  return threshold, precision, recall, f1, best_f1, area_pr

def main():
  fn = compute_edges_dxdy
  imlist = get_imlist('val')

  output_dir = 'output/demo'
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  bench_dir = os.path.join('output/demo', 'bench')
  if not os.path.exists(bench_dir):
    os.makedirs(bench_dir)

  print('Running detector:')
  runtime = detect_edges(imlist, fn, bench_dir)

  print('Evaluating:')
  sample_results, threshold_results, overall_result = \
    evaluate_boundaries.pr_evaluation(N_THRESHOLDS, imlist,
                                      load_gt_boundaries,
                                      lambda x: load_pred(bench_dir, x),
                                      fast=True, progress=tqdm)
  print('Save results:')
  out_file_name = os.path.join('output/demo', 'metrics.npz')
  threshold, precision, recall, f1, best_f1, area_pr = \
    save_results(out_file_name, threshold_results, overall_result, runtime)

  fig = plt.figure(figsize=(6,6))
  ax = fig.gca()
  display_results(ax, 'output/demo', threshold, precision, recall, f1, best_f1, area_pr)
  print('{:>24s}: {:<10.6f}'.format('runtime (in seconds)', runtime))

  fig.savefig(os.path.join(output_dir + '_pr.pdf'), bbox_inches='tight')

#if __name__ == '__main__':
 # app.run(main)


In [26]:
main()

<ipython-input-25-576415080106>:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return imlist.astype(np.int)


Running detector:


100%|██████████| 50/50 [00:00<00:00, 92.57it/s]


Evaluating:


100%|██████████| 50/50 [02:04<00:00,  2.49s/it]

Save results:
               threshold: 0.140000  
    overall max F1 score: 0.518496  
    average max F1 score: 0.556084  
                      AP: 0.426090  
    runtime (in seconds): 0.005991  


In [30]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import shutil

# Source folder in Colab workspace
source_folder = '/content/Lab2'

# Destination folder in Google Drive
destination_folder = '/content/drive/MyDrive/Lab2_LAK'

# Copy the entire folder to Google Drive
shutil.copytree(source_folder, destination_folder)


'/content/drive/MyDrive/Lab2_LAK'